In [2]:
from filmscope.reconstruction import RunManager, generate_config_dict 
from filmscope.util import get_timestamp
from filmscope.recon_util import get_all_sample_names
from filmscope.config import log_folder
from tqdm import tqdm
from matplotlib import pyplot as plt
import os
import torch

In [3]:
# make a config dict with a random sample name
# select the name of a sample previously saved using "save_new_sample.ipynb",
# the gpu number, and whether or not to log with neptune.ai
sample_name = "ant_photron"
gpu_number = "2"
use_neptune = True

os.environ["CUDA_VISIBLE_DEVICES"] = gpu_number

config_dict = generate_config_dict(sample_name=sample_name, gpu_number=gpu_number, downsample=1,
                                   camera_set="custom", use_neptune=use_neptune,
                                   run_args={"iters": 1000, "batch_size": 12, "num_depths": 64,
                                             "display_freq": 100},
                                   custom_image_numbers=[0, 1, 2],
                                   loss_weights={"smooth": 5, "smooth_lambda": 0}
                                   )

In [4]:
run_manager = RunManager(config_dict)

/home/clare/anaconda3/envs/filmscope/lib/python3.8/site-packages/torch/functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1724789115405/work/aten/src/ATen/native/TensorShape.cpp:3609.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
[neptune] [warning] NeptuneWarning: By default, these monitoring options are disabled in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', 'capture_hardware_metrics'. You can set them to 'True' when initializing the run and the monitoring will continue until you call run.stop() or the kernel stops. NOTE: To track the source files, pass their paths to the 'source_code' argument. For help, see: https://docs.neptune.ai/logging/source_code/


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/fourier-lightfield/FLCAM-MVS/e/FLCAM1-4841


In [5]:

# perform reconstruction
iters = config_dict["run_args"]["iters"]
reference_image = run_manager.dataset.reference_image.cpu().squeeze()
losses = []
for i in tqdm(range(iters)):
    log = (i % config_dict["run_args"]["display_freq"] == 0) or (i == iters - 1)
    mask_images, warp_images, numbers, outputs, loss_values = run_manager.run_epoch(
        i, log=(log and config_dict["use_neptune"]))
    losses.append(float(loss_values["total"]))

    # this section can be edited to change what is recorded
    #if log and not config_dict["use_neptune"]:

    # logging can also be performed with neptune or saving elsewhere
    # but this displays outputs with matplotlib
    if i % config_dict["run_args"]["display_freq"] == 0 and not config_dict["use_neptune"]:
        fig, (ax0, ax1) = plt.subplots(1, 2)
        ax1.imshow(outputs["depth"].detach().cpu().squeeze(), cmap='turbo')
        ax0.imshow(reference_image, cmap='gray')
        fig.suptitle(f"epoch {i}")
        plt.tight_layout()
        plt.show()

        plt.figure()
        plt.plot(losses)
        plt.xlabel("iteration")
        plt.ylabel("loss")
        plt.title("loss")
        plt.show()



100%|██████████| 1000/1000 [00:19<00:00, 51.27it/s]


In [ ]:
a = outputs["depth"].cpu().detach().squeeze()

In [ ]:
plt.imshow(a)#, clim=(-0.8, -0.2))
plt.colorbar()